<a href="https://colab.research.google.com/github/Brandt-DSTI/Computer_Vision_ISIC_2024/blob/main/Copy_of_EN_B0_BES_v_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Chunk 0: Mount Google Drive and Copy Files
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
drive_hdf5_path = '/content/drive/MyDrive/isic-2024-challenge/train-image.hdf5'
drive_metadata_path = '/content/drive/MyDrive/isic-2024-challenge/train-metadata.csv'
local_hdf5_path = '/content/train-image.hdf5'
local_metadata_path = '/content/train-metadata.csv'

# Copy files from Drive to local Colab session
shutil.copy(drive_hdf5_path, local_hdf5_path)
shutil.copy(drive_metadata_path, local_metadata_path)

print("Files copied to local Colab session.")
print(f"Train HDF5 file exists: {os.path.exists(local_hdf5_path)}")
print(f"Train metadata file exists: {os.path.exists(local_metadata_path)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files copied to local Colab session.
Train HDF5 file exists: True
Train metadata file exists: True


In [ ]:
# Chunk 1: Setup and Imports

# Install required packages
!pip install albumentations timm tqdm h5py opencv-python-headless imbalanced-learn mealpy

# Imports
import os
import random
import time
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import logging

from PIL import Image
import io
import h5py
import cv2

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau
from torch.amp import autocast, GradScaler

from mealpy import FloatVar, BES
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, f1_score, roc_curve, auc

import timm
from imblearn.over_sampling import RandomOverSampler

import albumentations as A
from albumentations.pytorch import ToTensorV2

import json
from datetime import datetime

# Setup logging
logging.basicConfig(filename='optimization.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Random seed setup for reproducibility
random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

# Load metadata
df_train = pd.read_csv(local_metadata_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 62.7 MB/s eta 0:00:00
Using device: cuda
GPU: NVIDIA A100-SXM4-40GB
Number of GPUs: 1


<ipython-input-2-15a17e1c4339>:65: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(local_metadata_path)


In [ ]:
# Chunk 2: Define Dataset and Augmentation

class ISICDataset(Dataset):
    def __init__(self, hdf5_file, isic_ids, targets=None, transform=None, vsurf_features=None):
        self.hdf5_file = h5py.File(hdf5_file, 'r')
        self.isic_ids = isic_ids
        self.targets = targets
        self.transform = transform
        self.vsurf_features = vsurf_features
        self.valid_indices = self._get_valid_indices()

    def _get_valid_indices(self):
        valid_indices = []
        for idx in range(len(self.isic_ids)):
            try:
                img_bytes = self.hdf5_file[self.isic_ids[idx]][()]
                Image.open(io.BytesIO(img_bytes))
                valid_indices.append(idx)
            except Exception as e:
                print(f"Error processing image at index {idx}: {e}")
        return valid_indices

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img_bytes = self.hdf5_file[self.isic_ids[real_idx]][()]
        img = Image.open(io.BytesIO(img_bytes))
        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']
        vsurf_feat = self.vsurf_features[real_idx] if self.vsurf_features is not None else np.zeros(5)
        target = self.targets[real_idx] if self.targets is not None else None
        return img, vsurf_feat, target

    def __del__(self):
        self.hdf5_file.close()

def get_augmentation(is_training=True):
    if is_training:
        return A.Compose([
            A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0)),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=5, min_height=8, min_width=8, fill_value=0, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(224, 224),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

In [ ]:
# Chunk 3: Train-Validation-Test Split, Random Oversampling, and DataLoader

def perform_oversampling(df_train, vsurf_features=None):
    X = df_train['isic_id'].values.reshape(-1, 1)
    y = df_train['target'].values
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X, y)

    resampled_vsurf_features = vsurf_features[ros.sample_indices_] if vsurf_features is not None else None

    resampled_df = pd.DataFrame({
        'isic_id': X_resampled.flatten(),
        'target': y_resampled
    })

    return resampled_df, resampled_vsurf_features

# Split data into train (70%), validation (15%), and test (15%)
train_df, temp_df = train_test_split(df_train, test_size=0.3, stratify=df_train['target'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['target'], random_state=42)

print(f"Train set: {len(train_df)} samples, Validation set: {len(val_df)} samples, Test set: {len(test_df)} samples")

def get_dataloaders(df, hdf5_file_path, batch_size, vsurf_features=None, is_training=True):
    dataset = ISICDataset(hdf5_file_path,
                          df['isic_id'].values,
                          df['target'].values,
                          transform=get_augmentation(is_training),
                          vsurf_features=vsurf_features)
    return DataLoader(dataset, batch_size=batch_size, shuffle=is_training, num_workers=4, pin_memory=True)

Train set: 280741 samples, Validation set: 60159 samples, Test set: 60159 samples


In [ ]:
# Chunk 4: Model Setup, Checkpointing Functions, and Objective Function with VSURF features

class EfficientNetWithVSURF(nn.Module):
    def __init__(self, model_name='efficientnet_b0', num_classes=1, vsurf_size=5, dropout_rate=0.5):
        super(EfficientNetWithVSURF, self).__init__()

        # Load the base model (EfficientNet)
        self.base_model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)

        # Replace classifier layer if necessary
        if 'efficientnet' in model_name:
            in_features = self.base_model.classifier.in_features
            self.base_model.classifier = nn.Identity()
        elif 'resnet' in model_name:
            in_features = self.base_model.fc.in_features
            self.base_model.fc = nn.Identity()

        # Adaptive pooling layer
        self.pool = nn.AdaptiveMaxPool2d((1, 1))

        # VSURF feature size
        self.vsurf_size = vsurf_size

        # Combined image + VSURF feature size
        combined_feature_size = in_features + self.vsurf_size

        # Fully connected layer
        self.fc = nn.Sequential(
            nn.Linear(combined_feature_size, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, num_classes)
        )

    def forward(self, img_inputs, vsurf_features):
        img_features = self.base_model(img_inputs)

        if len(img_features.size()) == 2:
            img_features = img_features.unsqueeze(-1).unsqueeze(-1)

        img_features = self.pool(img_features)
        img_features = img_features.view(img_features.size(0), -1)

        # Ensure vsurf_features have the same dtype as img_features
        vsurf_features = vsurf_features.to(img_features.dtype)

        combined_features = torch.cat([img_features, vsurf_features], dim=1)

        output = self.fc(combined_features)

        return output

def setup_model(model_name='efficientnet_b0', num_classes=1, dropout_rate=0.5, vsurf_size=5):
    model = EfficientNetWithVSURF(model_name=model_name, num_classes=num_classes, vsurf_size=vsurf_size, dropout_rate=dropout_rate)
    return model.to(device)

def save_checkpoint(epoch, model, optimizer, learning_rate, batch_size, dropout_rate):
    os.makedirs('bes_checkpoints', exist_ok=True)

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'learning_rate': learning_rate,
        'batch_size': batch_size,
        'dropout_rate': dropout_rate
    }
    checkpoint_path = f"bes_checkpoints/bes_checkpoint_epoch_{epoch}.pth"
    torch.save(checkpoint, checkpoint_path)
    logging.info(f"Checkpoint saved: {checkpoint_path}")

def load_latest_checkpoint():
    if not os.path.exists("bes_checkpoints"):
        logging.info("Checkpoint directory does not exist, starting from scratch.")
        return None

    checkpoints = [f for f in os.listdir("bes_checkpoints") if f.startswith("bes_checkpoint_epoch_")]
    if not checkpoints:
        logging.info("No checkpoints found, starting from scratch.")
        return None

    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    checkpoint_path = os.path.join("bes_checkpoints", latest_checkpoint)
    checkpoint = torch.load(checkpoint_path)

    model = setup_model(model_name='efficientnet_b0', num_classes=1, dropout_rate=checkpoint['dropout_rate'])
    model.load_state_dict(checkpoint['model_state_dict'])

    optimizer = Adam(model.parameters(), lr=checkpoint['learning_rate'], weight_decay=1e-5)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    logging.info(f"Loaded checkpoint: {checkpoint_path}")
    return checkpoint, model, optimizer

def train_and_evaluate_model(model, optimizer, scheduler, train_loader, val_loader, num_epochs=10, patience=5):
    criterion = nn.BCEWithLogitsLoss()
    scaler = GradScaler()
    best_val_f1 = 0
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, vsurf_features, targets in train_loader:
            images = images.to(device)
            vsurf_features = vsurf_features.to(device).float()  # Ensure float type
            targets = targets.to(device).float().view(-1, 1)

            optimizer.zero_grad()
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(images, vsurf_features)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        val_f1 = evaluate_model(model, val_loader)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val F1: {val_f1:.4f}")

        scheduler.step(val_f1)

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            epochs_without_improvement = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break

    return best_val_f1

@torch.no_grad()
def evaluate_model(model, val_loader):
    model.eval()
    all_targets = []
    all_outputs = []

    for images, vsurf_features, targets in val_loader:
        images = images.to(device)
        vsurf_features = vsurf_features.to(device).float()  # Ensure float type
        outputs = model(images, vsurf_features)

        all_outputs.extend(torch.sigmoid(outputs).cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

    binary_outputs = (np.array(all_outputs) > 0.5).astype(int)
    val_f1 = f1_score(all_targets, binary_outputs)

    return val_f1

def log_hyperparameters(hyperparameters, metrics, model_name, dataset_info):
    log = {
        "timestamp": datetime.now().isoformat(),
        "model": model_name,
        "dataset": dataset_info,
        "hyperparameters": hyperparameters,
        "performance_metrics": metrics
    }

    # Save as JSON for easy reading and sharing
    with open('hyperparameters_log.json', 'w') as f:
        json.dump(log, f, indent=4)

    # Also save as .pth for PyTorch compatibility
    torch.save(log, 'hyperparameters_log.pth')

def objective_function(solution):
    try:
        learning_rate, batch_size, dropout_rate = solution
        batch_size = int(batch_size)

        logging.info(f"Trying solution: LR={learning_rate:.6f}, BS={batch_size}, DR={dropout_rate:.2f}")

        train_df, val_df = train_test_split(df_train, test_size=0.2, stratify=df_train['target'], random_state=42)

        vsurf_columns = ['clin_size_long_diam_mm', 'tbp_lv_H', 'tbp_lv_deltaLBnorm', 'tbp_lv_perimeterMM', 'tbp_lv_Hext']
        vsurf_features = df_train[vsurf_columns].values

        df_train_resampled, resampled_vsurf_features = perform_oversampling(train_df, vsurf_features=vsurf_features[train_df.index])

        train_loader = get_dataloaders(df_train_resampled, local_hdf5_path, batch_size, vsurf_features=resampled_vsurf_features)
        val_loader = get_dataloaders(val_df, local_hdf5_path, batch_size, vsurf_features=vsurf_features[val_df.index], is_training=False)

        model = setup_model(model_name='efficientnet_b0', num_classes=1, dropout_rate=dropout_rate, vsurf_size=5)
        optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

        best_val_f1 = train_and_evaluate_model(model, optimizer, scheduler, train_loader, val_loader, num_epochs=3)

        # Prepare data for logging
        best_hyperparameters = {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'dropout_rate': dropout_rate
        }

        metrics = {
            'val_f1': best_val_f1,
            # Add other metrics if available, e.g.:
            # 'val_auc': val_auc,
            # 'val_pauc': val_pauc
        }

        dataset_info = {
            "name": "ISIC 2024 Challenge",
            "total_samples": len(df_train),
            "benign_samples": len(df_train[df_train['target'] == 0]),
            "malignant_samples": len(df_train[df_train['target'] == 1])
        }

        # Log hyperparameters
        log_hyperparameters(best_hyperparameters, metrics, "EfficientNet-B0", dataset_info)

        # Save the current best solution (keep this for backwards compatibility)
        current_best = {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'dropout_rate': dropout_rate,
            'val_f1': best_val_f1
        }
        torch.save(current_best, 'current_best_hyperparameters.pth')

        logging.info(f"Solution performance: F1={best_val_f1:.4f}")

        return -best_val_f1  # We're minimizing, so return negative F1 score

    except Exception as e:
        logging.error(f"Error in objective function: {str(e)}")
        return float('inf')  # Return large value to indicate failure


In [ ]:
# Chunk 5: BES Optimization Setup

# Define the problem dictionary
problem_dict = {
    "obj_func": objective_function,
    "bounds": FloatVar(
        lb=[1e-5, 16, 0.1],  # Lower bounds: learning_rate, batch_size, dropout_rate
        ub=[1e-3, 128, 0.5], # Upper bounds: learning_rate, batch_size, dropout_rate
        name=["learning_rate", "batch_size", "dropout_rate"]
    ),
    "minmax": "min",  # We want to minimize the objective function (negative F1 score)
}

# BES Optimizer Setup
optimizer = BES.OriginalBES(
    epoch=50,
    pop_size=10,
    a_factor=10,
    R_factor=1.5,
    alpha=2.0,
    c1=2.0,
    c2=2.0
)

# Set a callback function to save checkpoints after each epoch
def checkpoint_callback(epoch, population):
    os.makedirs('bes_checkpoints', exist_ok=True)
    checkpoint = {
        'epoch': epoch,
        'population': population
    }
    checkpoint_path = f"bes_checkpoints/bes_checkpoint_epoch_{epoch}.pth"
    torch.save(checkpoint, checkpoint_path)
    logging.info(f"Checkpoint saved: {checkpoint_path}")

optimizer.callback = checkpoint_callback

# Define termination conditions
term_dict = {
    "max_epoch": 50,
    "max_fe": 1000,
    "max_time": 36000,  # 10 hours in seconds
    "max_early_stop": 30
}

# Check for an existing checkpoint to resume optimization
latest_checkpoint = load_latest_checkpoint()
if latest_checkpoint:
    start_epoch = latest_checkpoint['epoch']
    optimizer.g_best = latest_checkpoint['g_best']
    optimizer.pop = latest_checkpoint['pop']
    optimizer.epoch = start_epoch
    logging.info(f"Resuming optimization from epoch {start_epoch}")
else:
    start_epoch = 0
    logging.info("Starting new optimization process")

# Solve the optimization problem using BES
try:
    best_solution, best_fitness = optimizer.solve(problem_dict, termination=term_dict)

    logging.info("Optimization completed successfully.")
    logging.info(f"Best Hyperparameters: Learning Rate = {best_solution[0]:.6f}, "
                 f"Batch Size = {int(best_solution[1])}, Dropout Rate = {best_solution[2]:.2f}")
    logging.info(f"Best Validation F1 Score: {-best_fitness:.4f}")

    # Output the results
    print("Optimization completed.")
    print(f"Best Hyperparameters: Learning Rate = {best_solution[0]:.6f}, Batch Size = {int(best_solution[1])}, "
          f"Dropout Rate = {best_solution[2]:.2f}")
    print(f"Best Validation F1 Score: {-best_fitness:.4f}")

except Exception as e:
    logging.error(f"Error during optimization process: {str(e)}")
    print(f"An error occurred during optimization. Check the log file for details.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/3, Train Loss: 0.0283, Val F1: 0.0748
Epoch 2/3, Train Loss: 0.0060, Val F1: 0.1017
Epoch 3/3, Train Loss: 0.0044, Val F1: 0.0000


INFO:mealpy.swarm_based.BES.OriginalBES:Solving single objective optimization problem.
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/3, Train Loss: 0.0375, Val F1: 0.0289
Epoch 2/3, Train Loss: 0.0117, Val F1: 0.0961
Epoch 3/3, Train Loss: 0.0092, Val F1: 0.0480


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/3, Train Loss: 0.0298, Val F1: 0.0567
Epoch 2/3, Train Loss: 0.0087, Val F1: 0.0828
Epoch 3/3, Train Loss: 0.0069, Val F1: 0.0545


KeyboardInterrupt: 

In [ ]:
# Chunk 6: Model evaluation with Kaggle pAUC scoring
import numpy as np
import torch
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, classification_report
from sklearn.metrics import f1_score as sklearn_f1_score

def competition_score(y_true, y_pred, min_tpr=0.80):
    # Rescale the target. Set 0s to 1s and 1s to 0s
    v_gt = abs(np.asarray(y_true) - 1)
    # Flip the predictions to their complements
    v_pred = -1.0 * np.asarray(y_pred)
    max_fpr = abs(1 - min_tpr)
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)  # Fixed syntax error here
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError(f"Expected min_tpr in range [0, 1), got: {min_tpr}")
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    return partial_auc, fpr, tpr

@torch.no_grad()
def evaluate_model(model, test_loader, device):
    model.eval()
    all_predictions = []
    all_targets = []
    for inputs, vsurf_features, targets in tqdm(test_loader, desc="Evaluating"):
        inputs = inputs.to(device)
        vsurf_features = vsurf_features.to(device)
        outputs = model(inputs, vsurf_features)
        predictions = torch.sigmoid(outputs).cpu().numpy()
        all_predictions.append(predictions)
        all_targets.append(targets.numpy())

    all_predictions = np.concatenate(all_predictions).flatten()
    all_targets = np.concatenate(all_targets)

    # Print diagnostic information
    print(f"Predictions - Min: {all_predictions.min():.4f}, Max: {all_predictions.max():.4f}, Mean: {all_predictions.mean():.4f}")
    print(f"Unique prediction values: {len(np.unique(all_predictions))}")
    print(f"Prediction distribution:\n{np.histogram(all_predictions, bins=10)}")
    print(f"Target distribution: {np.bincount(all_targets)}")

    # Calculate metrics
    try:
        auc_score = roc_auc_score(all_targets, all_predictions)
        pauc_score, fpr, tpr = competition_score(all_targets, all_predictions, min_tpr=0.80)
        print(f"ROC curve points: {len(fpr)}")
        print(f"TPR range: {tpr.min():.4f} to {tpr.max():.4f}")
    except Exception as e:
        print(f"Warning: AUC calculation failed. Error: {str(e)}")
        print("Setting AUC and pAUC to minimum values.")
        auc_score = 0.5
        pauc_score = 0.0
        fpr, tpr = None, None

    binary_predictions = (all_predictions > 0.5).astype(int)
    f1 = sklearn_f1_score(all_targets, binary_predictions)  # Use the imported function

    print(f"AUC: {auc_score:.4f}")
    print(f"pAUC (competition metric): {pauc_score:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_targets, binary_predictions))
    print("\nClassification Report:")
    print(classification_report(all_targets, binary_predictions))

    return auc_score, pauc_score, f1

# Load and evaluate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = setup_model(model_name='efficientnet_b0', num_classes=1, dropout_rate=0.5)
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)
print("Model loaded successfully.")

# Prepare the test dataset
test_df = df_train.sample(frac=0.2, random_state=42)  # Using 20% of the data as a test set
test_loader = get_dataloaders(test_df, '/content/train-image.hdf5', batch_size=64, is_training=False)

# Evaluate the model
auc_score, pauc_score, f1_score = evaluate_model(model, test_loader, device)

print(f"Best Model Test AUC: {auc_score:.4f}")
print(f"Best Model Test pAUC (competition metric): {pauc_score:.4f}")
print(f"Best Model Test F1 Score: {f1_score:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

<ipython-input-7-0b7d9299caea>:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth', map_location=device))


Model loaded successfully.


Evaluating:   0%|          | 0/1254 [00:00<?, ?it/s]

Predictions - Min: 0.0000, Max: 1.0000, Mean: 0.3095
Unique prediction values: 79082
Prediction distribution:
(array([43881,  4948,  3011,  2370,  2016,  2039,  2114,  2390,  3289,
       14154]), array([2.7602036e-07, 1.0000025e-01, 2.0000023e-01, 3.0000019e-01,
       4.0000015e-01, 5.0000012e-01, 6.0000008e-01, 7.0000011e-01,
       8.0000007e-01, 9.0000004e-01, 1.0000000e+00], dtype=float32))
Target distribution: [80138    74]
ROC curve points: 1106
TPR range: 0.0000 to 0.9996
AUC: 0.9806
pAUC (competition metric): 0.1807
F1 Score: 0.0061

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.70      0.82     80138
           1       0.00      0.99      0.01        74

    accuracy                           0.70     80212
   macro avg       0.50      0.84      0.42     80212
weighted avg       1.00      0.70      0.82     80212


Classification Report:
              precision    recall  f1-score   support

           0       1.

In [ ]:
# Chunk 7: Test against validation set
import traceback

# Prepare the validation dataset
vsurf_columns = ['clin_size_long_diam_mm', 'tbp_lv_H', 'tbp_lv_deltaLBnorm', 'tbp_lv_perimeterMM', 'tbp_lv_Hext']
val_vsurf_features = val_df[vsurf_columns].values

print("Validation VSURF features shape:", val_vsurf_features.shape)
print("Validation VSURF features type:", type(val_vsurf_features))
print("Validation VSURF features dtype:", val_vsurf_features.dtype)

# Create a DataLoader for the validation set
val_loader = get_dataloaders(val_df, '/content/train-image.hdf5', batch_size=64, vsurf_features=val_vsurf_features, is_training=False)

# Make sure your model is in evaluation mode
model.eval()

# Evaluate the model on the validation set
print("Evaluating on Validation Set:")
try:
    for batch_idx, (inputs, vsurf_features, targets) in enumerate(val_loader):
        print(f"\nBatch {batch_idx}:")
        print("Inputs shape:", inputs.shape)
        print("VSURF features shape:", vsurf_features.shape)
        print("Targets shape:", targets.shape)

        inputs = inputs.to(device)
        vsurf_features = vsurf_features.to(device)

        try:
            outputs = model(inputs, vsurf_features)
            print("Outputs shape:", outputs.shape)
        except Exception as e:
            print(f"Error in model forward pass: {str(e)}")
            print(f"Traceback: {traceback.format_exc()}")

        if batch_idx == 0:  # Only print for the first batch
            break

    val_auc_score, val_pauc_score, val_f1_score = evaluate_model(model, val_loader, device)
    print(f"Validation AUC: {val_auc_score:.4f}")
    print(f"Validation pAUC (competition metric): {val_pauc_score:.4f}")
    print(f"Validation F1 Score: {val_f1_score:.4f}")
except Exception as e:
    print(f"Error during validation evaluation: {str(e)}")
    print(f"Traceback: {traceback.format_exc()}")

# Print model structure
print("\nModel structure:")
print(model)


Validation VSURF features shape: (60159, 5)
Validation VSURF features type: <class 'numpy.ndarray'>
Validation VSURF features dtype: float64
Evaluating on Validation Set:

Batch 0:
Inputs shape: torch.Size([64, 3, 224, 224])
VSURF features shape: torch.Size([64, 5])
Targets shape: torch.Size([64])
Outputs shape: torch.Size([64, 1])


Evaluating:   0%|          | 0/940 [00:00<?, ?it/s]

Predictions - Min: 0.0000, Max: 1.0000, Mean: 0.0016
Unique prediction values: 59899
Prediction distribution:
(array([60004,    30,    21,    13,    14,     7,    12,    13,    12,
          33]), array([1.1199375e-09, 1.0000000e-01, 2.0000000e-01, 3.0000001e-01,
       4.0000001e-01, 5.0000000e-01, 6.0000002e-01, 6.9999999e-01,
       8.0000001e-01, 8.9999998e-01, 1.0000000e+00], dtype=float32))
Target distribution: [60100    59]
ROC curve points: 505
TPR range: 0.0000 to 0.8266
AUC: 0.9161
pAUC (competition metric): 0.1334
F1 Score: 0.3235

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60100
           1       0.29      0.37      0.32        59

    accuracy                           1.00     60159
   macro avg       0.64      0.69      0.66     60159
weighted avg       1.00      1.00      1.00     60159


Classification Report:
              precision    recall  f1-score   support

           0       1.0